In [1]:
import tensorflow as tf
#import tensorflow.nn.rnn_cell as rnn
import numpy as np
#import tensorflow.keras as keras
#from tensorflow.keras.initializers import Constant
import tensorflow.keras.backend as K


from tqdm import tqdm
import os


In [2]:
os.environ["CUDA_VISIBLE_DEVICES"]="-1"
re_train=True
time_steps=1
n_input =6
num_units=64
n_classes=1
lr =0.000001
logFile = './retrain/traing.txt'

In [4]:
data_x=np.loadtxt('./PSDSData (17)_raw.csv',delimiter=",")
data_y=np.loadtxt('./push(17).csv',delimiter=",")
print(data_y.sum())
print(len(data_y)/data_y.sum())
preY = np.loadtxt('./push(17).csv',delimiter=",")
data_x=data_x[:,3:9];
data_x.shape,data_y.shape

2424.0
7.439356435643564


((18033, 6), (18033,))

In [5]:
data_x.shape

(18033, 6)

In [6]:
datay=data_y
data_y=data_y.reshape(len(data_y),1)
data_x=data_x[:,(6-n_input):6].reshape((len(data_x),n_input))
data_state = np.zeros([1,num_units*2],dtype='float')
data_x.shape,data_y.shape

((18033, 6), (18033, 1))

In [7]:
# y [11111] to [10000]
def detect(predicts,raw,windows=16,threshold=0.8,raw_threshold=-10):
    taps=np.zeros_like(predicts)
    windows_count=0
    buff_size=25
    
    for i in np.arange(buff_size,len(taps)):
        raw_buff=raw[i-buff_size:i+1,2]# only z acc
        #print(raw_buff.min())
        diff=abs(raw_buff.max()-raw_buff.min())
        buff_sum = 0
        for x in raw_buff:
            if x <0:
                buff_sum+= x*-1
        
        if predicts[i]>threshold and windows_count==0  and buff_sum>raw_threshold:# and diff>6:# and predicts[i-1]>threshold:
            #print(diff)
            taps[i]=1
            windows_count=windows
            #print(buff_sum)
        else:
            windows_count=max(windows_count-1,0)
    return taps

In [8]:
taps_label = detect(data_y,raw=data_x)

In [9]:
taps_label.shape

(18033, 1)

In [10]:
loss_weight = datay*len(datay)/datay.sum()+1
loss_weight = loss_weight.reshape([len(loss_weight),1])

In [11]:
x= tf.placeholder('float',[None,n_input],name='input_tensor')
y = tf.placeholder('float',[None,n_classes],name='labels')
#inputs = tf.unstack(x,time_steps,1,name='input_tensor')
#state_h = tf.placeholder('float',[None,num_units],name='input_h')
#state_c = tf.placeholder('float',[None,num_units],name='input_c')
state_in   = tf.placeholder('float',[None,num_units*2],name='input_state')
loss_weight_in = tf.placeholder('float',[None,1],name='loss_weight')

In [12]:
wi=tf.Variable(name='wi',initial_value=tf.random_normal([n_input, num_units], stddev=0.02))
wf=tf.Variable(name='wf',initial_value=tf.random_normal([n_input, num_units], stddev=0.02))
wc=tf.Variable(name='wc',initial_value=tf.random_normal([n_input, num_units], stddev=0.02))
wo=tf.Variable(name='wo',initial_value=tf.random_normal([n_input, num_units], stddev=0.02))
ui=tf.Variable(name='ui',initial_value=tf.random_normal([num_units, num_units], stddev=0.02))
uf=tf.Variable(name='uf',initial_value=tf.random_normal([num_units, num_units], stddev=0.02))
uc=tf.Variable(name='uc',initial_value=tf.random_normal([num_units, num_units], stddev=0.02))
uo=tf.Variable(name='uo',initial_value=tf.random_normal([num_units, num_units], stddev=0.02))

bi=tf.Variable(name='bi',initial_value=tf.random_normal([num_units], stddev=0.02))
bf=tf.Variable(name='bf',initial_value=tf.random_normal([num_units], stddev=0.02))
bc=tf.Variable(name='bc',initial_value=tf.random_normal([num_units], stddev=0.02))
bo=tf.Variable(name='bo',initial_value=tf.random_normal([num_units], stddev=0.02))

out_weights1=tf.Variable(name="weights1",initial_value=tf.random_normal([num_units, 16], stddev=0.02))
out_weights2=tf.Variable(name="weights2",initial_value=tf.random_normal([16, 1], stddev=0.02))
out_bias1=tf.Variable(name="bias1",initial_value=tf.random_normal([16], stddev=0.02))
out_bias2=tf.Variable(name="bias2",initial_value=tf.random_normal([1], stddev=0.02))

In [13]:
state_h,state_c = tf.split(state_in,num_or_size_splits=2,axis=1)
x_i = K.dot(x,wi)
x_f = K.dot(x,wf)
x_c = K.dot(x,wc)
x_o = K.dot(x,wo)

i = K.hard_sigmoid(tf.add(tf.add(x_i,bi),K.dot(state_h,ui)))
f = K.hard_sigmoid(tf.add(tf.add(x_f,bf),K.dot(state_h,uf)))
c = tf.add(tf.multiply(f,state_c),tf.multiply(i,K.tanh(tf.add(tf.add(x_c,bc),K.dot(state_h,uc)))),name='output_c')
o = K.hard_sigmoid(tf.add(tf.add(x_o,bo),K.dot(state_h,uo)))
h = tf.multiply(o,tf.tanh(c),name='output_h')

In [14]:
dense1  = tf.nn.relu(tf.add(tf.matmul(h,out_weights1), out_bias1))
prediction=tf.nn.relu(tf.add(tf.matmul(dense1,out_weights2), out_bias2,name='output_forRelu'), name="output")
#prediction=tf.nn.relu(tf.add(tf.matmul(h,out_weights), out_bias,name='output_forRelu'), name="output")
state_out = tf.concat([h,c],axis=1,name='output_state')
prediction_weight=tf.matmul(prediction,loss_weight_in)
y_weight=tf.matmul(y,loss_weight_in)

loss=tf.losses.mean_squared_error(predictions=prediction_weight,labels=y_weight)
opt=tf.train.AdamOptimizer(learning_rate=lr).minimize(loss,name="train_op")
init = tf.global_variables_initializer()
saver=tf.train.Saver()
sess = tf.Session()
sess.run(init)

W0923 09:26:07.066440 139797338949440 deprecation.py:323] From /home/xing/anaconda3/envs/tf_cpu/lib/python3.6/site-packages/tensorflow/python/ops/losses/losses_impl.py:121: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [15]:
opt.name

'train_op'

In [16]:
def verify(taps,label_taps):
    False_positive= np.zeros_like(taps)
    False_negtive = np.zeros_like(taps)
    True_positive = np.zeros_like(taps)
    window=15
    #taps=np.append(np.zeros([1]),taps)
    #taps=np.append(taps,np.zeros([1]))
    #label_taps=np.append(np.zeros([1]),label_taps)
    #label_taps=np.append(label_taps,np.zeros([1]))
    for i in range(len(taps)):
        if label_taps[i] ==1:
            if taps[i-window:i+window].sum()==0:
                False_negtive[i]=1
            else:
                True_positive[i]=1

        if taps[i] ==1:
            if label_taps[i-window:i+window].sum()==0:
                False_positive[i]=1
                #raw_buff=raw[i-4:i+1,2]# only z acc
                #diff=abs(raw_buff.max()-raw_buff.min())
                #print(i,raw_buff,diff)
            
    return [False_positive,  False_negtive ,    True_positive ]

In [17]:
iters=1
predictY=np.zeros([len(data_x)])
while iters<10:
    for i in tqdm(range(len(data_x))):
        feed={x:data_x[i:i+1],
              state_in:data_state,
                y:data_y[i:i+1],
              loss_weight_in:loss_weight[i:i+1]
               }
        if re_train:
            sess.run(opt,feed_dict=feed)

        data_state,_predictY = sess.run([state_out,prediction],feed_dict=feed)
        #print(data_h.shape)
        #data_c = sess.run(c,feed_dict=feed)
        predictY[i]=_predictY[0]
        #print(i)
    #print(sess.run('bi:0'))
    saver.save(sess,'./retrain/test_model'+str(iters-1))
    if iters%1==0:
        print(iters)
        mse = (np.square(predictY-data_y.reshape([len(data_x)]))).mean(axis=0)
        loss_weight = loss_weight.reshape([len(loss_weight)])
        mse_weight =np.square(np. multiply((predictY-data_y.reshape([len(data_x)])),loss_weight)).mean(axis=0)
        loss_weight = loss_weight.reshape([len(loss_weight),1])
        taps_predict = detect(predictY,raw=data_x)
        FP,FN,TP = verify(taps_predict,taps_label)
        file = open(logFile,'a+')
        file.write('\niters:'+str(iters))
        file.write('\nmse:'+str(mse))
        file.write('\nmse_weight:'+str(mse_weight))
        file.write('\ntotal taps:'+str(taps_predict.sum()))
        file.write('\nFP,FN,TP:'+str(FP.sum())+', '+str(FN.sum())+', '+str(TP.sum()))
        file.close()
        print('mse:'+str(mse))
        print('mse_weight:'+str(mse_weight))
        print('total taps:'+str(taps_predict.sum()))
        print('FP,FN,TP:'+str(FP.sum())+', '+str(FN.sum())+', '+str(TP.sum()))
        if iters%10==0:

            loss_weight = predictY*len(predictY)/predictY.sum()+datay*len(datay)/datay.sum()+1
            loss_weight = loss_weight.reshape([len(loss_weight),1])
            print('noneTapWeight'+str(len(predictY)/predictY.sum()))

    output_graph='./retrain/'+str(num_units)+'_i'+str(iters)+'_mse'+'{0:.2f}'.format(mse*100)+'_'+'{0:.2f}'.format(mse_weight)+'.pb'
    #output_graph='../models/100hz/realLSTM_data_U'+str(num_units)+'_i'+str(iters)+'_mse'+'{0:.2f}'.format(mse*100)+'_'+'{0:.2f}'.format(mse_weight)+'.pb'
    output_graph_def = tf.compat.v1.graph_util.convert_variables_to_constants(
            sess, # The session is used to retrieve the weights
            tf.get_default_graph().as_graph_def(), # The graph_def is used to retrieve the nodes 
            ['output','output_state'] # The output node names are used to select the usefull nodes
        ) 
    with tf.gfile.GFile(output_graph, "wb") as f:
            f.write(output_graph_def.SerializeToString())
    iters=iters+1
    out_preY = output_graph.split('.pb')[0]+'_preY.csv'
    np.savetxt(out_preY,predictY)
    #saver.save(sess,'./retrain/test_model'+str(iters-1))

100%|██████████| 18033/18033 [00:23<00:00, 769.19it/s]


1


W0923 09:27:20.436162 139797338949440 deprecation.py:323] From <ipython-input-17-f42e7c7bfc87>:50: convert_variables_to_constants (from tensorflow.python.framework.graph_util_impl) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.compat.v1.graph_util.convert_variables_to_constants`
W0923 09:27:20.436730 139797338949440 deprecation.py:323] From /home/xing/anaconda3/envs/tf_cpu/lib/python3.6/site-packages/tensorflow/python/framework/graph_util_impl.py:270: extract_sub_graph (from tensorflow.python.framework.graph_util_impl) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`
  0%|          | 14/18033 [00:00<02:09, 138.90it/s]

mse:0.12356915429324036
mse_weight:8.664564849656294
total taps:0.0
FP,FN,TP:0.0, 202.0, 0.0


100%|██████████| 18033/18033 [00:22<00:00, 809.90it/s]


...

8


  0%|          | 31/18033 [00:00<00:58, 309.60it/s]

mse:0.6143877977635938
mse_weight:0.7231424295612817
total taps:915.0
FP,FN,TP:551.0, 2.0, 200.0


100%|██████████| 18033/18033 [00:21<00:00, 857.84it/s]


9
mse:0.5910468294628695
mse_weight:0.7217132931284309
total taps:844.0
FP,FN,TP:496.0, 4.0, 198.0


In [17]:
data_state.shape

(1, 128)

In [19]:
save1=saver

AttributeError: 'Saver' object has no attribute 'max_to_keep'

In [17]:
#Load model and train new data

with tf.Session() as sess1:
    init=tf.global_variables_initializer()
    sess1.run(init)
    new_saver =tf.train.import_meta_graph('./retrain/test_model1.meta')
    new_saver.restore(sess1,('./retrain/test_model1'))
    print(sess1.run('bi:0'))
    graph=tf.get_default_graph()
    #print([n.name for n in tf.get_default_graph().as_graph_def().node])
    train_step=graph.get_operation_by_name("train_op")
    for i in tqdm(range(len(data_x))):
        feed={"input_tensor:0":data_x[i:i+1],
              "input_state:0":data_state,
              "labels:0":data_y[i:i+1],
              "loss_weight:0":loss_weight[i:i+1]
               }
        sess1.run(train_step,feed_dict=feed)
        data_state,_predictY = sess1.run(["output_state:0","output:0"],feed_dict=feed)
    print(sess1.run('bi:0'))

W0913 14:08:56.287348 140538466105152 deprecation.py:323] From /home/xing/anaconda3/envs/tf_cpu/lib/python3.6/site-packages/tensorflow/python/training/saver.py:1276: checkpoint_exists (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
  0%|          | 0/18033 [00:00<?, ?it/s]

[-5.7063447e-03 -1.8465047e-03  5.8838739e-03 -6.1452705e-03
  3.9387941e-02  3.4139113e-04 -1.5409730e-02  3.5584066e-03
  3.5744114e-03 -9.5328046e-03  9.9528451e-03  2.3483027e-02
 -3.9534951e-03 -2.4520522e-02  7.1736299e-03 -3.8554464e-02
  1.8904410e-02 -9.8872939e-03  1.7602913e-02 -1.1894713e-03
  2.3886776e-02  9.2592156e-03  8.2292957e-03  7.3925760e-03
  4.1925203e-02 -7.2104442e-03  3.6520809e-02 -2.4253873e-02
  4.5465745e-02 -9.7077349e-03  1.5023111e-03  7.5401845e-03
 -2.1253533e-03  2.1378510e-02 -2.4253979e-02  6.6657639e-03
 -4.4258814e-03 -1.8435629e-02  7.0998478e-03 -2.6672983e-03
  1.4627862e-02  6.3955672e-03  9.3487650e-03  6.3213904e-04
  3.8069315e-02  3.9136343e-02  1.4842294e-02  1.4239924e-03
  9.3876934e-03 -2.3107868e-02 -1.3689954e-05 -3.5314772e-03
  5.8002626e-03 -2.9823719e-02 -9.2941336e-03  2.3730097e-02
 -6.7457925e-03 -2.5603618e-03  4.0934734e-02  1.3285197e-04
  2.1441130e-02  4.0641176e-03  1.5266694e-03  3.6570849e-03]


100%|██████████| 18033/18033 [00:24<00:00, 750.88it/s]

[ 5.4655288e-04  4.1932571e-03  1.2069216e-02  8.3020983e-05
  4.5332953e-02  6.5130554e-03 -9.0284301e-03  9.9591156e-03
  9.4893007e-03 -3.2852038e-03  1.6295318e-02  2.9366689e-02
  2.3052709e-03 -1.8435707e-02  1.3196019e-02 -3.5097402e-02
  2.5160553e-02 -3.9449562e-03  2.4069609e-02  5.1611145e-03
  3.0340910e-02  1.5648670e-02  1.4800979e-02  1.3480432e-02
  4.8062161e-02 -1.0244285e-03  4.2536229e-02 -1.7964175e-02
  5.1538408e-02 -3.3480572e-03  8.1804972e-03  1.3606368e-02
  3.8716404e-03  2.7285727e-02 -1.8231349e-02  1.3117496e-02
  1.8132577e-03 -1.2090015e-02  1.2996495e-02  3.4741813e-03
  2.0607755e-02  1.2896574e-02  1.5563586e-02  6.6715190e-03
  4.4332627e-02  4.5326903e-02  2.1079680e-02  7.3527810e-03
  1.5505044e-02 -1.6964620e-02  5.9772595e-03  2.6771771e-03
  1.2224833e-02 -2.3821309e-02 -3.1242734e-03  2.9864591e-02
 -5.6090078e-04  3.7588375e-03  4.7067113e-02  5.7254382e-03
  2.7678646e-02  1.0284196e-02  7.7630761e-03  1.0018184e-02]


In [44]:
data_state